In [ ]:
!git clone https://github.com/jishengpeng/WavTokenizer

In [2]:
%cd WavTokenizer

/kaggle/working/WavTokenizer


In [6]:
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt

--2025-04-09 16:16:34--  https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.61, 3.165.160.59, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f8/d8/f8d8b97f33126a1e3a4c3ffe2e6af86c26776cfa33aee1294672329b62115562/078d11581aa10cc91572bfcff7ef00b71d8e24c4c359e98f9194a1a9d21ae8a8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27wavtokenizer_medium_music_audio_320_24k_v2.ckpt%3B+filename%3D%22wavtokenizer_medium_music_audio_320_24k_v2.ckpt%22%3B&Expires=1744218995&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDIxODk5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y4L2Q4L2Y4ZDhiOTdmMzMxMjZhMWUzYTRjM2ZmZTJlNmFmODZjMjY3NzZjZmEzM2FlZTEyOTQ2NzIzM

In [7]:
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml

--2025-04-09 16:17:07--  https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.59, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2995 (2.9K) [text/plain]
Saving to: ‘wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml’

wavtokenizer_medium 100%[===================>]   2.92K  --.-KB/s    in 0s      

2025-04-09 16:17:07 (1.02 GB/s) - ‘wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml’ saved [2995/2995]



In [8]:
from encoder.utils import convert_audio
import torchaudio
import torch
from decoder.pretrained import WavTokenizer


device=torch.device('cuda')

config_path = "/kaggle/working/WavTokenizer/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml"
model_path = "/kaggle/working/WavTokenizer/wavtokenizer_medium_music_audio_320_24k_v2.ckpt"

wavtokenizer = WavTokenizer.from_pretrained0802(config_path, model_path)
wavtokenizer = wavtokenizer.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/kaggle/working/WavTokenizer/decoder/pretrained.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True

## Groove dataset

In [34]:
wav_files = []

for root, dirs, files in os.walk('/kaggle/input/groove1/groove/'):
    for file in files:
        if file.endswith('.wav'):
            wav_files.append(os.path.join(root, file))

In [59]:
! rm -rf /kaggle/working/groove

In [60]:
from tqdm import tqdm

for wav_file in tqdm(wav_files):
    wav, sr = torchaudio.load(wav_file)
    wav = convert_audio(wav, sr, 24000, 1) 
    bandwidth_id = torch.tensor([0])
    wav=wav.to(device)
    _,discrete_code= wavtokenizer.encode_infer(wav, bandwidth_id=bandwidth_id)

    name = wav_file.split('/')[-1][:-3] + 'pt'
    new_path = os.path.join('/kaggle/working/', *wav_file.split('/')[4:-1])
    os.makedirs(new_path, exist_ok = True)
    torch.save(discrete_code, os.path.join(new_path, name))

100%|██████████| 1090/1090 [05:13<00:00,  3.48it/s]


In [71]:
import shutil
shutil.make_archive('groove_tokens', 'zip', '/kaggle/working/groove')

'/kaggle/working/WavTokenizer/groove_tokens.zip'

## MusicNet dataset

In [78]:
wav_files = []

for root, dirs, files in os.walk('/kaggle/input/musicnet-dataset/'):
    for file in files:
        if file.endswith('.wav'):
            wav_files.append(os.path.join(root, file))

In [88]:
! rm -rf /kaggle/working/musicnet

In [92]:
from tqdm import tqdm

for wav_file in tqdm(wav_files):
    wav, sr = torchaudio.load(wav_file)
    wav = convert_audio(wav, sr, 24000, 1) 
    bandwidth_id = torch.tensor([0])
    wav=wav.to(device)
    if wav.shape[-1] > 6e6:
        wavs = torch.split(wav, 6000000, -1)
    
        for i, small_wav in enumerate(wavs):
            _,discrete_code= wavtokenizer.encode_infer(small_wav, bandwidth_id=bandwidth_id)
        
            name = wav_file.split('/')[-1][:-4] + f'_{i}' + '.pt'
            new_path = os.path.join('/kaggle/working/', *wav_file.split('/')[4:-1])
            os.makedirs(new_path, exist_ok = True)
            torch.save(discrete_code, os.path.join(new_path, name))

    else:
        _,discrete_code= wavtokenizer.encode_infer(wav, bandwidth_id=bandwidth_id)

        name = wav_file.split('/')[-1][:-3] + 'pt'
        new_path = os.path.join('/kaggle/working/', *wav_file.split('/')[4:-1])
        os.makedirs(new_path, exist_ok = True)
        torch.save(discrete_code, os.path.join(new_path, name))

100%|██████████| 330/330 [15:20<00:00,  2.79s/it]


In [93]:
import shutil
shutil.make_archive('musicnet_tokens', 'zip', '/kaggle/working/musicnet')

'/kaggle/working/WavTokenizer/musicnet_tokens.zip'